# Building Web Applications with Gradio

This tutorial will guide you through using Gradio to build interactive web applications from your Python functions. We'll cover Gradio's design philosophy, main components, and deployment options.

## What is Gradio?

Gradio is an open-source Python library that makes it easy to create customizable UI components for machine learning models, APIs, or any Python function. It's designed to make sharing your work as simple as possible.

### Design Philosophy

Gradio follows several key principles:

1. **Simplicity First**: Convert any Python function into a web interface with just a few lines of code
2. **Zero Web Development Knowledge Required**: No HTML, CSS, or JavaScript needed
3. **Rapid Prototyping**: Get a working demo in minutes, not hours
4. **Flexible Components**: Rich set of input/output components for different data types
5. **Easy Sharing**: Built-in sharing capabilities and deployment options

### When to Use Gradio

Gradio is particularly well-suited for:
- Machine learning model demos
- Data analysis tools
- Image/audio processing applications
- API testing interfaces
- Educational demonstrations
- Quick prototypes and proof-of-concepts

## Installation and Setup

First, let's install Gradio and import the necessary libraries:

In [2]:
import gradio as gr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import base64

## Core Concepts

### 1. The Basic Pattern

Every Gradio app follows this pattern:
1. Define a Python function
2. Create an interface with `gr.Interface()`
3. Launch the app with `.launch()`

Let's start with the classic "Hello World" example:

In [3]:
def greet(name):
    return f"Hello, {name}!"

# Create the interface
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text",
    title="Hello World App",
    description="Enter your name to get a greeting!"
)

# Launch the app
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### 2. Input and Output Components

Gradio provides a rich set of components for different data types:

In [6]:
def calculator(operation, num1, num2):
    if operation == "Add":
        return num1 + num2
    elif operation == "Subtract":
        return num1 - num2
    elif operation == "Multiply":
        return num1 * num2
    elif operation == "Divide":
        if num2 != 0:
            return num1 / num2
        else:
            return "Error: Division by zero!"

calculator_demo = gr.Interface(
    fn=calculator,
    inputs=[
        gr.Dropdown(["Add", "Subtract", "Multiply", "Divide"], label="Operation"),
        gr.Number(label="First Number"),
        gr.Number(label="Second Number")
    ],
    outputs=gr.Number(label="Result"),
    title="Simple Calculator",
    description="Perform basic arithmetic operations"
)

calculator_demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863
* Running on public URL: https://dc1ed76f57f50d3789.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Main Components

### Input Components

- **Text**: `gr.Textbox()` - for text input
- **Number**: `gr.Number()` - for numeric input
- **Slider**: `gr.Slider()` - for selecting values within a range
- **Dropdown**: `gr.Dropdown()` - for selecting from options
- **Checkbox**: `gr.Checkbox()` - for boolean input
- **Radio**: `gr.Radio()` - for selecting one option
- **Image**: `gr.Image()` - for image upload
- **Audio**: `gr.Audio()` - for audio upload
- **File**: `gr.File()` - for file upload

### Output Components

- **Text**: `gr.Textbox()` - for text output
- **Label**: `gr.Label()` - for classification results
- **Image**: `gr.Image()` - for image output
- **Plot**: `gr.Plot()` - for matplotlib plots
- **HTML**: `gr.HTML()` - for HTML content
- **JSON**: `gr.JSON()` - for JSON data
- **Dataframe**: `gr.Dataframe()` - for tabular data

## Practical Examples

### Example 1: Data Analysis Tool

Let's create a data analysis tool using the penguins dataset:

In [4]:
# Load the penguins dataset
penguins = pd.read_csv('data/penguins.csv')

def analyze_penguins(species, plot_type):
    # Filter data if specific species selected
    if species != "All":
        data = penguins[penguins['species'] == species]
    else:
        data = penguins
    
    # Create plot
    plt.figure(figsize=(10, 6))
    
    if plot_type == "Body Mass Distribution":
        if species == "All":
            sns.histplot(data=data, x='body_mass_g', hue='species', bins=20)
        else:
            sns.histplot(data=data, x='body_mass_g', bins=20)
        plt.title(f'Body Mass Distribution - {species}')
        plt.xlabel('Body Mass (g)')
        
    elif plot_type == "Bill Length vs Depth":
        if species == "All":
            sns.scatterplot(data=data, x='bill_length_mm', y='bill_depth_mm', hue='species')
        else:
            sns.scatterplot(data=data, x='bill_length_mm', y='bill_depth_mm')
        plt.title(f'Bill Length vs Depth - {species}')
        plt.xlabel('Bill Length (mm)')
        plt.ylabel('Bill Depth (mm)')
    
    plt.tight_layout()
    
    # Return both plot and summary statistics
    summary = data.describe()
    
    return plt.gcf(), summary

# Get unique species for dropdown
species_options = ["All"] + list(penguins['species'].unique())

penguin_analyzer = gr.Interface(
    fn=analyze_penguins,
    inputs=[
        gr.Dropdown(species_options, value="All", label="Species"),
        gr.Radio(["Body Mass Distribution", "Bill Length vs Depth"], 
                value="Body Mass Distribution", label="Plot Type")
    ],
    outputs=[
        gr.Plot(label="Visualization"),
        gr.Dataframe(label="Summary Statistics")
    ],
    title="Penguin Data Explorer",
    description="Explore penguin data with interactive visualizations"
)

penguin_analyzer.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


### Example 2: Image Processing Tool

Let's create a simple image processing application:

In [ ]:
def process_image(image, brightness, contrast, rotate):
    if image is None:
        return None
    
    # Convert to PIL Image if needed
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)
    
    # Apply brightness adjustment
    from PIL import ImageEnhance
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(brightness)
    
    # Apply contrast adjustment
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(contrast)
    
    # Apply rotation
    if rotate != 0:
        image = image.rotate(rotate, expand=True)
    
    return image

image_processor = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(label="Upload Image"),
        gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="Brightness"),
        gr.Slider(0.1, 2.0, value=1.0, step=0.1, label="Contrast"),
        gr.Slider(-180, 180, value=0, step=15, label="Rotation (degrees)")
    ],
    outputs=gr.Image(label="Processed Image"),
    title="Image Processor",
    description="Upload an image and adjust brightness, contrast, and rotation"
)

image_processor.launch()

### Example 3: Text Analysis Tool

Let's create a text analysis application:

In [7]:
import re
from collections import Counter

def analyze_text(text):
    if not text:
        return "Please enter some text to analyze.", {}, ""
    
    # Basic statistics
    word_count = len(text.split())
    char_count = len(text)
    sentence_count = len(re.findall(r'[.!?]+', text))
    paragraph_count = len([p for p in text.split('\n\n') if p.strip()])
    
    # Word frequency
    words = re.findall(r'\w+', text.lower())
    word_freq = Counter(words)
    top_words = dict(word_freq.most_common(10))
    
    # Summary statistics
    stats = f"""
    📊 Text Analysis Results:
    
    • Word Count: {word_count:,}
    • Character Count: {char_count:,}
    • Sentence Count: {sentence_count:,}
    • Paragraph Count: {paragraph_count:,}
    • Average Words per Sentence: {word_count/max(sentence_count, 1):.1f}
    • Average Characters per Word: {char_count/max(word_count, 1):.1f}
    """
    
    # Create word frequency chart
    if top_words:
        plt.figure(figsize=(10, 6))
        words_list = list(top_words.keys())
        counts_list = list(top_words.values())
        
        plt.bar(words_list, counts_list)
        plt.title('Top 10 Most Frequent Words')
        plt.xlabel('Words')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        chart = plt.gcf()
    else:
        chart = None
    
    return stats, top_words, chart

text_analyzer = gr.Interface(
    fn=analyze_text,
    inputs=gr.Textbox(
        lines=10, 
        placeholder="Enter your text here for analysis...",
        label="Text Input"
    ),
    outputs=[
        gr.Textbox(label="Statistics"),
        gr.JSON(label="Top 10 Words"),
        gr.Plot(label="Word Frequency Chart")
    ],
    title="Text Analyzer",
    description="Analyze text for word count, frequency, and other statistics",
    examples=[
        ["This is a sample text for analysis. It contains multiple sentences and demonstrates various text analysis features."],
        ["Machine learning is a subset of artificial intelligence. It focuses on algorithms that can learn from data."]
    ]
)

text_analyzer.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## Advanced Features

### 1. Multiple Inputs and Outputs

Gradio can handle functions with multiple inputs and outputs:

In [8]:
def data_summary_and_plot(csv_file, column_name, plot_type):
    if csv_file is None:
        return "Please upload a CSV file", None, None
    
    try:
        df = pd.read_csv(csv_file.name)
        
        if column_name not in df.columns:
            return f"Column '{column_name}' not found in the dataset", None, df.head()
        
        # Generate summary
        summary = f"""
        Dataset Summary:
        • Shape: {df.shape[0]} rows × {df.shape[1]} columns
        • Column '{column_name}' statistics:
          - Mean: {df[column_name].mean():.2f}
          - Median: {df[column_name].median():.2f}
          - Std Dev: {df[column_name].std():.2f}
          - Min: {df[column_name].min():.2f}
          - Max: {df[column_name].max():.2f}
        """
        
        # Generate plot
        plt.figure(figsize=(10, 6))
        if plot_type == "Histogram":
            plt.hist(df[column_name].dropna(), bins=20, alpha=0.7)
            plt.title(f'Distribution of {column_name}')
        elif plot_type == "Box Plot":
            plt.boxplot(df[column_name].dropna())
            plt.title(f'Box Plot of {column_name}')
        
        plt.xlabel(column_name)
        plt.tight_layout()
        
        return summary, plt.gcf(), df.head(10)
        
    except Exception as e:
        return f"Error processing file: {str(e)}", None, None

csv_analyzer = gr.Interface(
    fn=data_summary_and_plot,
    inputs=[
        gr.File(label="Upload CSV File", file_types=[".csv"]),
        gr.Textbox(label="Column Name", placeholder="Enter column name for analysis"),
        gr.Radio(["Histogram", "Box Plot"], value="Histogram", label="Plot Type")
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.Plot(label="Visualization"),
        gr.Dataframe(label="Data Preview")
    ],
    title="CSV Data Analyzer",
    description="Upload a CSV file and analyze a specific column"
)

csv_analyzer.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


### 2. Using Gradio Blocks for Custom Layouts

For more control over the layout, you can use `gr.Blocks()`:

In [9]:
def simple_math(operation, a, b):
    if operation == "+":
        return a + b
    elif operation == "-":
        return a - b
    elif operation == "*":
        return a * b
    elif operation == "/":
        return a / b if b != 0 else "Division by zero!"

with gr.Blocks(title="Advanced Calculator") as calculator_blocks:
    gr.Markdown("# 🧮 Advanced Calculator")
    gr.Markdown("Perform mathematical operations with a custom interface")
    
    with gr.Row():
        with gr.Column():
            num1 = gr.Number(label="First Number", value=0)
            operation = gr.Dropdown(["+", "-", "*", "/"], label="Operation", value="+")
            num2 = gr.Number(label="Second Number", value=0)
            calculate_btn = gr.Button("Calculate", variant="primary")
        
        with gr.Column():
            result = gr.Number(label="Result")
            gr.Markdown("### Quick Examples")
            gr.Examples(
                examples=[
                    ["+", 10, 5],
                    ["-", 10, 3],
                    ["*", 7, 8],
                    ["/", 15, 3]
                ],
                inputs=[operation, num1, num2]
            )
    
    calculate_btn.click(
        fn=simple_math,
        inputs=[operation, num1, num2],
        outputs=result
    )

calculator_blocks.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


## Best Practices

### 1. Error Handling

Always handle potential errors in your functions:

```python
def safe_function(input_data):
    try:
        # Your processing logic
        result = process_data(input_data)
        return result
    except Exception as e:
        return f"Error: {str(e)}"
```

### 2. Input Validation

Validate inputs before processing:

```python
def validate_and_process(text, number):
    if not text or not text.strip():
        return "Please provide valid text input"
    
    if number < 0:
        return "Number must be positive"
    
    return process_data(text, number)
```

### 3. Performance Optimization

For heavy computations:
- Use caching for expensive operations
- Implement progress bars for long-running tasks
- Consider using `gr.Interface(..., live=True)` for real-time updates

### 4. User Experience

- Provide clear descriptions and labels
- Use examples to guide users
- Add helpful titles and descriptions
- Consider mobile responsiveness

## Challenge: Multi-Feature Data Dashboard

Let's create a comprehensive dashboard that combines multiple features:

In [10]:
# Load sample data
penguins = pd.read_csv('data/penguins.csv')
car_crashes = pd.read_csv('data/car_crashes.csv')

def create_dashboard(dataset, analysis_type, species_filter=None):
    try:
        if dataset == "Penguins":
            data = penguins.copy()
            if species_filter and species_filter != "All":
                data = data[data['species'] == species_filter]
        else:
            data = car_crashes.copy()
        
        # Create visualization based on analysis type
        plt.figure(figsize=(12, 8))
        
        if analysis_type == "Overview":
            # Create subplot with multiple charts
            fig, axes = plt.subplots(2, 2, figsize=(15, 10))
            
            if dataset == "Penguins":
                # Species distribution
                data['species'].value_counts().plot(kind='bar', ax=axes[0,0])
                axes[0,0].set_title('Species Distribution')
                axes[0,0].tick_params(axis='x', rotation=45)
                
                # Body mass distribution
                data['body_mass_g'].hist(bins=20, ax=axes[0,1])
                axes[0,1].set_title('Body Mass Distribution')
                
                # Correlation heatmap
                numeric_cols = data.select_dtypes(include=[np.number]).columns
                sns.heatmap(data[numeric_cols].corr(), annot=True, ax=axes[1,0])
                axes[1,0].set_title('Correlation Matrix')
                
                # Scatter plot
                sns.scatterplot(data=data, x='bill_length_mm', y='bill_depth_mm', 
                              hue='species', ax=axes[1,1])
                axes[1,1].set_title('Bill Length vs Depth')
            
            plt.tight_layout()
            dashboard_plot = plt.gcf()
        
        elif analysis_type == "Statistical Summary":
            # Return statistical summary
            summary = data.describe()
            dashboard_plot = None
        
        # Generate insights
        insights = f"""
        📊 Dataset: {dataset}
        📏 Shape: {data.shape[0]} rows × {data.shape[1]} columns
        🔍 Analysis: {analysis_type}
        
        Key Insights:
        • Total records analyzed: {len(data):,}
        • Missing values: {data.isnull().sum().sum()}
        • Numeric columns: {len(data.select_dtypes(include=[np.number]).columns)}
        • Categorical columns: {len(data.select_dtypes(include=[object]).columns)}
        """
        
        return insights, dashboard_plot, data.head(10), data.describe()
        
    except Exception as e:
        return f"Error: {str(e)}", None, None, None

# Create the dashboard interface
with gr.Blocks(title="Data Science Dashboard", theme=gr.themes.Soft()) as dashboard:
    gr.Markdown("# 📊 Interactive Data Science Dashboard")
    gr.Markdown("Explore datasets with interactive visualizations and analysis")
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 🎛️ Controls")
            dataset_choice = gr.Radio(
                ["Penguins", "Car Crashes"], 
                value="Penguins", 
                label="Dataset"
            )
            analysis_choice = gr.Radio(
                ["Overview", "Statistical Summary"], 
                value="Overview", 
                label="Analysis Type"
            )
            species_choice = gr.Dropdown(
                ["All"] + list(penguins['species'].unique()), 
                value="All", 
                label="Species Filter (Penguins only)",
                visible=True
            )
            analyze_btn = gr.Button("🔍 Analyze Data", variant="primary")
        
        with gr.Column(scale=2):
            gr.Markdown("### 📈 Results")
            insights_output = gr.Textbox(label="Insights", lines=8)
            plot_output = gr.Plot(label="Visualization")
    
    with gr.Row():
        data_preview = gr.Dataframe(label="Data Preview")
        stats_summary = gr.Dataframe(label="Statistical Summary")
    
    # Event handlers
    def update_species_visibility(dataset):
        return gr.update(visible=(dataset == "Penguins"))
    
    dataset_choice.change(
        fn=update_species_visibility,
        inputs=dataset_choice,
        outputs=species_choice
    )
    
    analyze_btn.click(
        fn=create_dashboard,
        inputs=[dataset_choice, analysis_choice, species_choice],
        outputs=[insights_output, plot_output, data_preview, stats_summary]
    )
    
    # Add examples
    gr.Examples(
        examples=[
            ["Penguins", "Overview", "Adelie"],
            ["Penguins", "Statistical Summary", "All"],
            ["Car Crashes", "Overview", None]
        ],
        inputs=[dataset_choice, analysis_choice, species_choice]
    )

dashboard.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


## Conclusion

Gradio provides a powerful and intuitive way to create web applications from Python functions. Key takeaways:

### ✅ Advantages of Gradio:
- **Simplicity**: Convert functions to web apps with minimal code
- **Rich Components**: Extensive input/output component library
- **Flexible Layouts**: Both simple interfaces and custom layouts
- **Easy Deployment**: Multiple deployment options including Hugging Face Spaces
- **Interactive**: Real-time updates and user interaction
- **Shareable**: Built-in sharing capabilities

### 🎯 Best Use Cases:
- Machine learning model demos
- Data analysis and visualization tools
- Image/audio processing applications
- Educational demonstrations
- Quick prototypes and proof-of-concepts
- API testing interfaces

### 🚀 Next Steps:
1. Practice with the examples in this notebook
2. Try deploying your apps to Hugging Face Spaces
3. Explore advanced features like custom CSS and JavaScript
4. Build applications for your specific use cases
5. Share your work with the community

### 📚 Additional Resources:
- [Official Gradio Documentation](https://gradio.app/docs/)
- [Gradio GitHub Repository](https://github.com/gradio-app/gradio)
- [Hugging Face Spaces](https://huggingface.co/spaces)
- [Gradio Community Examples](https://huggingface.co/spaces?search=gradio)

Happy building! 🎉